# measure the average minimum distance between two lines

In [20]:
from geopandas import read_file
from shapely.geometry import Point
import numpy as np

# 5 m resolution

In [4]:
least_cost_path_5_false = read_file(r"E:\Basti\Studium\HSHarz\WissenschaftlichesArbeiten\WebProcessing-StandardisedGISAnalysesforCableRoutePlanning\results\least_cost_paths\least_cost_path_test_points_res_5_al_false.gpkg")
least_cost_path_5_true = read_file(r"E:\Basti\Studium\HSHarz\WissenschaftlichesArbeiten\WebProcessing-StandardisedGISAnalysesforCableRoutePlanning\results\least_cost_paths\least_cost_path_test_points_res_5_al_true.gpkg")

least_cost_path_5_false.geometry.values[0].distance(least_cost_path_5_true.geometry.values[0])

0.0

In [7]:
least_cost_path_5_false.geometry.values[0].length, least_cost_path_5_true.geometry.values[0].length

(76136.27072666152, 78002.0066579491)

In [33]:
distances_5  =  np.array([Point(p).distance(least_cost_path_5_true.geometry.values[0]) for p in least_cost_path_5_false.geometry.values[0].coords])

In [34]:
mean_min_distances_5 =  np.mean(distances_5)
mean_min_distances_5

126.04115368394045

# 10 m resolution

In [25]:
least_cost_path_10_false = read_file(r"E:\Basti\Studium\HSHarz\WissenschaftlichesArbeiten\WebProcessing-StandardisedGISAnalysesforCableRoutePlanning\results\least_cost_paths\least_cost_path_test_points_res_10_al_false.gpkg")
least_cost_path_10_true = read_file(r"E:\Basti\Studium\HSHarz\WissenschaftlichesArbeiten\WebProcessing-StandardisedGISAnalysesforCableRoutePlanning\results\least_cost_paths\least_cost_path_test_points_res_10_al_true.gpkg")
least_cost_path_10_false.geometry.values[0].length, least_cost_path_10_true.geometry.values[0].length

(75430.10086958889, 77936.56874848259)

In [35]:
distances_10  =  np.array([Point(p).distance(least_cost_path_10_true.geometry.values[0]) for p in least_cost_path_10_false.geometry.values[0].coords])
mean_min_distances_10 =  np.mean(distances_10)
mean_min_distances_10

277.921414632231

# 50 m resolution

In [28]:
least_cost_path_50_false = read_file(r"E:\Basti\Studium\HSHarz\WissenschaftlichesArbeiten\WebProcessing-StandardisedGISAnalysesforCableRoutePlanning\results\least_cost_paths\least_cost_path_test_points_res_50_al_false.gpkg")
least_cost_path_50_true = read_file(r"E:\Basti\Studium\HSHarz\WissenschaftlichesArbeiten\WebProcessing-StandardisedGISAnalysesforCableRoutePlanning\results\least_cost_paths\least_cost_path_test_points_res_50_al_true.gpkg")
least_cost_path_50_false.geometry.values[0].length, least_cost_path_50_true.geometry.values[0].length

(76135.0213422174, 70619.94558270913)

In [36]:
distances_50  =  np.array([Point(p).distance(least_cost_path_50_true.geometry.values[0]) for p in least_cost_path_50_false.geometry.values[0].coords])
mean_min_distances_50 =  np.mean(distances_50)
mean_min_distances_50

1140.0069288754526

# 100 m resolution

In [30]:
least_cost_path_100_false = read_file(r"E:\Basti\Studium\HSHarz\WissenschaftlichesArbeiten\WebProcessing-StandardisedGISAnalysesforCableRoutePlanning\results\least_cost_paths\least_cost_path_test_points_res_100_al_false.gpkg")
least_cost_path_100_true = read_file(r"E:\Basti\Studium\HSHarz\WissenschaftlichesArbeiten\WebProcessing-StandardisedGISAnalysesforCableRoutePlanning\results\least_cost_paths\least_cost_path_test_points_res_100_al_true.gpkg")
least_cost_path_100_false.geometry.values[0].length, least_cost_path_100_true.geometry.values[0].length

(76283.80711525407, 74120.73210305478)

In [37]:
distances_100  =  np.array([Point(p).distance(least_cost_path_100_true.geometry.values[0]) for p in least_cost_path_100_false.geometry.values[0].coords])
mean_min_distances_100 =  np.mean(distances_100)
mean_min_distances_100

1946.410723103447

# Distances all touched False

In [38]:
distances_false  =  np.array([Point(p).distance(least_cost_path_100_false.geometry.values[0]) for p in least_cost_path_5_false.geometry.values[0].coords])
mean_min_distances_false =  np.mean(distances_false)
mean_min_distances_false

257.9733268785573

# Distances all touched True


In [39]:
distances_true  =  np.array([Point(p).distance(least_cost_path_100_true.geometry.values[0]) for p in least_cost_path_5_true.geometry.values[0].coords])
mean_min_distances_true =  np.mean(distances_true)
mean_min_distances_true

2109.436322189748